In [105]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)
import json
import markup
import natasha

from bs4 import BeautifulSoup as Soup
import chardet
import unicodedata
import re

In [106]:
def get_raw_text(path_to_file):
    with open(path_to_file, 'rb') as f:
        rtext = f.read()
        rtext_unicode = rtext.decode(chardet.detect(rtext)["encoding"]).encode("utf8").decode("utf8")
        text = unicodedata.normalize("NFKD", rtext_unicode)
        soup = Soup(text[text.find('<body>'):], features="lxml")

    without_strong = []
    for offer in soup.find_all('p'):
        if '<strong>' not in str(offer) and '<emphasis>' not in str(offer):
            without_strong.append(offer.get_text(strip=True))
    text = ' '.join(without_strong[2:])
    return text


In [109]:
text = get_raw_text('Данные/Не обработанные/А. Белый/belyj_a-text_0040.fb2')

In [110]:
def get_doc(text):
    doc = Doc(text)

    segmenter = Segmenter()
    doc.segment(segmenter)

    emb = NewsEmbedding()
    tagger = NewsMorphTagger(emb)
    doc.tag_morph(tagger)

    morph_vocab = MorphVocab()
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return doc

In [111]:
# функция get_doc от get_raw достает doc-и для других функицй
doc = get_doc(get_raw_text('Данные/Не обработанные/А. Белый/belyj_a-text_0040.fb2'))

In [112]:
doc.tokens

[DocToken(stop=8, text='Источник', pos='NOUN', feats=<Inan,Nom,Masc,Sing>, lemma='источник'),
 DocToken(start=8, stop=9, text=':', pos='PUNCT', lemma=':'),
 DocToken(start=10, stop=17, text='Андрей', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='андрей'),
 DocToken(start=18, stop=24, text='Белый', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='белый'),
 DocToken(start=24, stop=25, text=';', pos='PUNCT', lemma=';'),
 DocToken(start=26, stop=27, text='"', pos='PUNCT', lemma='"'),
 DocToken(start=27, stop=36, text='Петербург', pos='PROPN', feats=<Inan,Nom,Masc,Sing>, lemma='петербург'),
 DocToken(start=36, stop=37, text='"', pos='PUNCT', lemma='"'),
 DocToken(start=37, stop=38, text=',', pos='PUNCT', lemma=','),
 DocToken(start=39, stop=44, text='Роман', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='роман'),
 DocToken(start=44, stop=45, text='.', pos='PUNCT', lemma='.'),
 DocToken(start=46, stop=52, text='Изд-во', pos='PROPN', feats=<Anim,Nom,Masc,Sing>, lemma='изд-во'),
 DocToke

In [101]:
import re
def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))


def get_sents(doc):
    sents_list = []
    for sent in doc.sents:
        sents_list.append(sent.text)

    return sents_list

def get_words(doc):
    only_words = []
    for token in doc.tokens:
        if has_cyrillic(token.text):
            only_words.append(token.text)
    return only_words

def get_lemma_words(doc):
    #lemma_words = {_.text: _.lemma for _ in doc.tokens}
    lemma_words = [token.lemma for token in doc.tokens]
    return lemma_words

def get_part_speech(doc, pos):
    part_speech = []
    for token in doc.tokens:
        if token.pos == pos:
            part_speech.append(token.text)
    return part_speech

def get_lemma_part_speech(doc, pos):
    lemma_part_speech = []
    for token in doc.tokens:
        if token.pos == pos:
            lemma_part_speech.append(token.lemma)
    return lemma_part_speech


In [104]:
doc.text

'Источник: Андрей Белый; "Петербург", Роман. Изд-во: "Кристалл", Санкт-Петербург, 1999. OCR и вычитка: Давид Титиевский (davidtit@land.ru), Хайфа, 28 января 2003. Корректура: Александр Белоусенко, 1 марта 2003.   Роман в восьми главах с прологом и эпилогом  Андрей Белый. Петербург / Роман.-- СПб. ООО "Издательство "Кристалл"", 1999.-- 976 с., илл.   Только камни нам дал чародей, Да Неву буро-желтого цвета, Да пустыни немых площадей, Где казнили людей до рассвета.  А что было у нас на земле, Чем вознесся орел наш двуглавый, В темных лаврах гигант на скале, -- Завтра станет ребячьей забавой.  На тебя надевали тиару -- юрода колпак, Бирюзовый учитель, мучитель, властитель, дурак! Как снежок по Москве, заводил кавардак гоголек, Непонятен--понятен, невнятен, запутан, легок. Конькобежец и первенец, веком гонимый взашей Под морозную пыль образуемых вновь падежей.   Мысль сама себя мыслит. Книга сама себя пишет: "оттуда, из этой вот точки, несется потоком рой отпечатанной кн